# Companies in Canada

- 1 million companies lines.

**Must:** 
1. ~~Business Name~~
2. ~~Primary Contact phone~~
3. ~~Mailing address~~
5. ~~Mailing city ~~
6. ~~Website~~
7. ~~Mailing Province~~
8. ~~Business activity~~
9. ~~Mailing Postal~~
8. ~~Industry~~ -- we hard code this.
9. ~~NAICS primary~~ -- we hard code this.
4. ~~Fax~~ == phone
11. Primary Contact name and title
13. Primary Contact email
14. Product /services

**Optional:** \
15. Number of employees \
16. ~~Physical address~~ \
17. ~~Physical city \~~
18. Year founded \
19. ~~Physical Province \~~
20. ~~Physical Postal \~~
21. Sales volume \
22. SIC primary 


**Missing**:
- Primary Contact email
- Primary Contact name
- Fax

- Suggestion: pull email from website.

In [1]:
import scrap as src

from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup, SoupStrainer
from urllib.parse import urlsplit
import urllib.request

import pandas as pd
import numpy as np

import os, sys, httplib2, json, fire, re, string, requests
from collections import OrderedDict, deque
import re, requests, requests.exceptions

import warnings
warnings.filterwarnings("ignore")


pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)

# Yellow Pages

In [ ]:
class yelllowPages():
    
    def __init__(self, url):
        self.url = url
    
    def fetchAttributes(self):
        
        # title
        titles = src.get_elements(self.url, 
                       search={"class": "listing__name jsMapBubbleName"},
                 tag='h3'
                )
        titles = [i for i in titles if i.isdigit() == False][:-1]
        
        # activity: revisit the length
        activities = src.get_elements(self.url, 
                 tag='article',
                 search = {"class": "listing__details__teaser"}
                )
        activities = [i for i in activities if i !="See more text"]
        activities = [i for i in activities if i !="more..."]
        
        # website: revisit length and alignment
        def get_link(url):
            parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
            resp = urllib.request.urlopen(url)
            soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

            links = []
            for link in soup.find_all('a', href=True):
                if "/gourl/" in link['href']:
                    links.append(link['href'])

            return links
        sites = get_link(self.url)
        sites = [i.split("%3A%2F%2F")[-1].split("%")[0] for i in sites]
        
        # physical & mailing addresses
        def address_in4s(url):
    
            parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
            resp = urllib.request.urlopen(url)
            soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))
            addresses = soup.find_all("span", class_="jsMapBubbleAddress")
            addresses = [addresses[i: i + 4] for i in range(0, len(addresses), 4)]

            return addresses
        
        addresses = address_in4s(self.url)
        listed = []
        for j in range(len(addresses)):
            listed.append([i.getText() for i in addresses[j]])

        Mailing_address = []
        Mailing_city = []
        Mailing_Province = []
        Mailing_Postal = []
        Physical_address = []
        Physical_city = []

        for i in range(len(listed)):
            Mailing_address.append(listed[i][0])
            Physical_address.append(listed[i][0])
            Mailing_city.append(listed[i][1])  
            Physical_city.append(listed[i][1])    
            Mailing_Province.append(listed[i][2])    
            Mailing_Postal.append(listed[i][3])   
            
        # phones + fax
        def get_phone(url):
            source_code = requests.get(url)
            plain_text = source_code.content
            soup = BeautifulSoup(plain_text, "lxml")
            links = soup.findAll('a', {'class': 'mlr__item__cta jsMlrMenu'})

            phones = []
            for link in links:
                title = link.get('data-phone')
                phones.append(title)

            return phones
    
        phones = get_phone(self.url)
        fax = phones
        
        # emails
        def get_email(url):

            unprocessed_urls = deque([url])
            processed_urls = set()
            emails = set()
            url = url

            print("Crawling URL %s" % url)
            response = requests.get(url)

            new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
            new_emails = [mail for mail in new_emails if ".png" not in mail]
            new_emails = [mail for mail in new_emails if ".jpg" not in mail]
            new_emails = [i for i in new_emails if len(i) < 30]

            emails.update(new_emails)

            return emails

        mails = []
        for site in [i.split("%3A%2F%2F")[-1].split("%")[0] for i in get_link(self.url)]:
            try:
                mail = get_email("http://" + site)

            except Exception:
                try:
                    mail = get_email("https://" + site)

                except Exception:
                    mail = "null"
                    pass

            mails.append(mail)
            
        # products and services
        pass
    
        return titles, activities, sites, phone, fax, mails, Mailing_address, Mailing_city,
    Mailing_Province, Mailing_Postal, Physical_address, Physical_city
    
    def create_df():
        df = pd.DataFrame(columns = [
            "name", "phone", "activity", "website", "mail", "mail_address",
            "mail_city", "mail_province", "fax", "mail_postal", "physical_address", "physical_city"
        ])
        df["name"], df["activity"], df["website"], df["phone"], df["fax"], df["name"], df["name"], 
        df["name"], df["name"], df["name"], df["name"], df["name"],  = fetchAttributes(self.url)

## Loop page nos

In [ ]:
for i in range(500):
    try:
        pass
    except Exception:
        pass
    
# add industry and NAICS

5. Position mixup.

In [25]:
# parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
# resp = urllib.request.urlopen("https://www.yellowpages.ca/search/si/1/Restaurants/Toronto+ON")
# soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

# links = []
# for link in soup.find_all('a', href=True):
#     if "bus/" in link['href']:
#         if "#" not in link["href"]:
#             if "Review" not in link["href"]:
#                 links.append(link['href'])



# links = list(OrderedDict.fromkeys(links))
# links